In [ ]:
%run ./00_Mapping_Setup

In [ ]:
/* ===================================================================================
   METRIC: EMP03 - Code of Conduct Timely Training Completion (Template Output)
=================================================================================== */

WITH Training_Data AS (
    SELECT 
        TRY_CAST(TRIM(DeptID) AS INT) AS Cleaned_DeptID,
        CASE 
            WHEN DueDate IS NULL OR TRIM(DueDate) = '' THEN 0
            WHEN CompletionDateUTC IS NULL OR TRIM(CompletionDateUTC) = '' THEN 0
            WHEN to_date(TRIM(CompletionDateUTC), 'M/d/yyyy') <= to_date(TRIM(DueDate), 'M/d/yyyy') THEN 1
            ELSE 0 
        END AS Completed_On_Time
    FROM hive_metastore.ra_adido_2025.td_code_of_conduct_and_ethics_110102024_10312025
    WHERE DeptID IS NOT NULL
),

Mapped_AUs AS (
    SELECT 
        m.AU_ID AS BusinessID,
        t.Completed_On_Time,
        CASE WHEN t.Cleaned_DeptID IS NOT NULL THEN 1 ELSE 0 END AS Has_Employee
    FROM vw_cost_center_mapping_bootstrap m
    LEFT JOIN Training_Data t
        ON CAST(m.Cost_Center_ID AS INT) = t.Cleaned_DeptID
)

SELECT 
    BusinessID,                          
    'EMP03' AS QuestionID,               
    
    CASE 
        WHEN SUM(Has_Employee) = 0 THEN '0%' 
        ELSE CONCAT(
            CAST(ROUND((SUM(Completed_On_Time) * 100.0) / SUM(Has_Employee), 2) AS STRING), 
            '%'
        )
    END AS Response
    
FROM Mapped_AUs
GROUP BY BusinessID
ORDER BY BusinessID;

In [ ]:
/* ===================================================================================
   METRIC: EMP03 - Troubleshooting View for Failed Training Records
=================================================================================== */

SELECT 
    m.AU_ID AS BusinessID,
    m.Cost_Center_ID,
    TRIM(t.DeptID) AS Raw_DeptID,
    t.DueDate,
    t.CompletionDateUTC,
    
    -- Dynamically prints out the exact reason the record failed the logic
    CASE 
        WHEN t.DueDate IS NULL OR TRIM(t.DueDate) = '' THEN 'Failed: Missing Due Date'
        WHEN t.CompletionDateUTC IS NULL OR TRIM(t.CompletionDateUTC) = '' THEN 'Failed: Missing Completion Date'
        WHEN to_date(TRIM(t.CompletionDateUTC), 'M/d/yyyy') > to_date(TRIM(t.DueDate), 'M/d/yyyy') THEN 'Failed: Completed Late'
        ELSE 'Unknown Error / Parse Failure'
    END AS Failure_Reason

FROM vw_cost_center_mapping_bootstrap m

JOIN hive_metastore.ra_adido_2025.td_code_of_conduct_and_ethics_110102024_10312025 t
    ON CAST(m.Cost_Center_ID AS INT) = TRY_CAST(TRIM(t.DeptID) AS INT)

-- Only return the rows that failed the EMP03 logic
WHERE 
    (t.DueDate IS NULL OR TRIM(t.DueDate) = '')
    OR (t.CompletionDateUTC IS NULL OR TRIM(t.CompletionDateUTC) = '')
    OR (to_date(TRIM(t.CompletionDateUTC), 'M/d/yyyy') > to_date(TRIM(t.DueDate), 'M/d/yyyy'))

ORDER BY BusinessID, Cost_Center_ID;